# Linear Ridge regression using Laplacian random features

In [1]:
import matplotlib.pyplot as plt
import torch
from torchmetrics.classification import MulticlassAccuracy, MulticlassCalibrationError
from utils import load_fmnist
from tqdm.notebook import trange
import torch.nn.functional as F
from torchkernels.feature_maps import LaplacianRFF

In [2]:
DEVICE = torch.device('cpu')
X_train, X_test, y_train, y_test = load_fmnist(device=DEVICE)
n, d = X_train.shape
c = len(y_train.unique())
y_train_onehot = F.one_hot(y_train, num_classes=c).float()
y_test_onehot = F.one_hot(y_test, num_classes=c).float()

### Define random feature map

In [3]:
p = 1024 # number of random features
feature_map = LaplacianRFF(input_dim=d, num_features=p, device = DEVICE)

### Training Setup

In [4]:
criterion = torch.nn.MSELoss()
acc = MulticlassAccuracy(num_classes=c)
ece = MulticlassCalibrationError(num_classes=c)
w = torch.zeros(p, c, requires_grad=True)
lam = 1e-4 # Regularization strength

In [5]:
Phi = feature_map(X_train)
w = torch.linalg.solve(Phi.T@Phi + lam*torch.eye(p), Phi.T@y_train_onehot)

y_hat_train = Phi@w
y_hat_test = feature_map(X_test)@w

## Metrics

In [6]:
print(f"Train accuracy:{' '*5}{acc(y_hat_train, y_train):.3f}")
print(f"Test accuracy:{' '*6}{acc(y_hat_test, y_test):.3f}")
print("--"*20)
print(f"Train ECE:{' '*10}{ece(y_hat_train, y_train):.3f}")
print(f"Test ECE:{' '*11}{ece(y_hat_test, y_test):.3f}")

Train accuracy:     0.863
Test accuracy:      0.842
----------------------------------------
Train ECE:          0.672
Test ECE:           0.653
